# Linear Congruential Generators



In [83]:
def prng_lcg(seed, repeat):
    multiplier = 5 # the "multiplier"
    increment = 3  # the "increment"
    modulus = 17  # the "modulus"
    state = seed
    file= open("LCG.rnd","w")

    for _ in range(repeat):
        state = (state * multiplier + increment) % modulus
        #print(str(state).zfill(19))
        file.write(str(state).zfill(19)+"\n")

In [84]:
prng_lcg(10, 200)

## LCG Cracking: Unknown increment



In [85]:
multiplier = 5 # the "multiplier"
increment = 3  # the "increment"
modulus = 17  # the "modulus"
rn = [0, 0, 0, 0, 0, 0]
file = open("LCG.rnd","r")
for i in range(6):
    rn[i] = int(file.readline())


In [86]:
def crack_unknown_increment(states, modulus, multiplier):
    [x0,x1, *_] = states
    increment = (x1- (multiplier* x0)) % modulus
    return increment


In [87]:
recovered_increment = crack_unknown_increment(rn, modulus, multiplier)
assert recovered_increment==increment
print("[x] Recovered increment : ", recovered_increment)

[x] Recovered increment :  3


## LCG Cracking: Unknown multiplier



In [88]:
import functools


def gcd(x, y): # GCD
    while(y): 
        x, y = y, x % y
    return x

def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, x, y = egcd(b % a, a)
        return (g, y - (b // a) * x, x)

def modinv(b, n):  # Modular inverse function
    g, x, _ = egcd(b, n)
    if g == 1:
        return x % n

In [89]:
def crack_unknown_multiplier(states, modulus):
    [x0,x1,x2, *_] = states
    mod_inv = modinv(x1-x0, modulus)
    multiplier = ((x2 - x1) * mod_inv) % modulus 
    increment = crack_unknown_increment(states, modulus, multiplier)
    return increment, multiplier

In [90]:
recovered_increment, recovered_multiplier = crack_unknown_multiplier(rn, modulus)
assert recovered_multiplier==multiplier
print("[x] Recovered multiplier : ", recovered_multiplier)

[x] Recovered multiplier :  5


## LCG Cracking: Unknown modulus

this time we can’t solve this with simple linear equations – since we don’t know modulus, so every equation we’ll form will introduce new unknown.

Fortunately, there is a mathematical trick that usually comes in handy in situations like this. Namely, interesting number theory fact

if we have few random multiples of n, with large probability their gcd will be equal to n.

if we can think of some modular operations that will give something congruent to zero, 

for example:$X \equiv 0~(\mod n)$

Then, by definition, this is equivalent to: $X = k*n$

This is only interesting if $X != 0$, but $X \equiv 0~(\mod n).$

We just need to take a gcd from few such values, and we can recover $n$. 

This is a really generic method, that can often be used when modulus that we’re using is unknown.



In [91]:
def crack_unknown_modulus(states):
    diffs = [s1 - s0 for s0, s1 in zip(states, states[1:])]
    zeroes = [t2*t0 - t1*t1 for t0, t1, t2 in zip(diffs, diffs[1:], diffs[2:])]
    modulus = abs(functools.reduce(gcd, zeroes))
    increment, multiplier = crack_unknown_multiplier(states, modulus)
    return modulus, increment, multiplier

In [92]:
recovered_modulus, recovered_increment, recovered_multiplier  = crack_unknown_modulus(rn)
assert recovered_modulus==modulus
print("[x] Recovered modulus : ", recovered_modulus)

[x] Recovered modulus :  17


**Exercise**

Assume you got a ciphertext \(in hexa\) that  is  62716b6b777862677362667e7b,  as a result of XOR encryption \(ASCII encoding is used\).

Additionally, assume that you know the original encrypted plaintext was 'cybersecurity'

You know that the key were random numbers generated by LCG. This means that each character in the plaintext has been encrypted \(XORed\) with a single generated random number of LCG.

By knowing the above information, crack the LCG parameters.



In [93]:
# Your code here
# 62716b6b777862677362667e7b
# cybersecurity

cipher_str = '62716b6b777862677362667e7b'
cipher_int = [int(cipher_str[i:i+2], 16) for i in range(0, len(cipher_str), 2)]
plain_str = 'cybersecurity'
plain_int = [ord(ch) for ch in plain_str ]
key_lcg = [l ^ r for l, r in zip(cipher_int, plain_int)]

# q = crack_unknown_modulus(key_lcg)
u_mod, u_inc, u_mul = crack_unknown_modulus(key_lcg)

print(f"{u_mod=}, {u_mul=}, {u_inc=}")

u_mod=17, u_mul=5, u_inc=3


# Mersenne Twister



In [94]:
'''
Mersenne Twister (MT) Cracker
Based on randcrack package
It uses random function, since random is based on MT.

Step 1: feed cracker with 624 randomly generated numbers (32-bit integers)
Step 2: After submitting 624 integers it will be ready for predicting new numbers.

'''

import random, time
from randcrack import RandCrack


#random.seed(time.time())
random.seed(100)

rc = RandCrack()

print("[x] feeding the cracker with 624 randomly generated numbers")
for i in range(624):
	r = random.getrandbits(32)
	rc.submit(r)

print("[x] Now, it is ready to predict new randomly generated numbers ..")
print("\n[x] Random Number: {}\n[x] Predicted number: {}"
	.format(random.randrange(0, 4294967295), rc.predict_randrange(0, 4294967295)))

[x] feeding the cracker with 624 randomly generated numbers
[x] Now, it is ready to predict new randomly generated numbers ..

[x] Random Number: 2143607628
[x] Predicted number: 2143607628
